# Angepasster Code zur Verarbeitung des Copernicus DEM GLO-30 Datensatzes zu den CAMELS-DE Daten zur Topografie

Da der direkte Webzugriff auf DEM Glo-30, wie er im Camels-Code genutzt wird, nicht mehr möglich ist, wird der Datensatz für den Bereich Deutschland manuell heruntergeladen und in den input_data Ordner gespeichert.

Der folgende Code-Block veranschaulicht das Auftreten der Null-Werte und kann zur direkten Berechnung auch übersprungen werden.

In [ ]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
import numpy as np
import matplotlib.pyplot as plt

from catchments import get_catchment_gdf

ID = 1

catchment_gdf = get_catchment_gdf(ID)

# Load the merged Copernicus DEM raster
print("Loading merged Copernicus DEM raster")
dem = rasterio.open(".../input_data/dem/dem_merged.tif")

# Setting the nodata value if not defined in the raster metadata
if dem.nodata is None:
    nodata_value = -9999
else:
    nodata_value = dem.nodata

print(f"DEM bounds: {dem.bounds}")
print(f"DEM CRS: {dem.crs}")

if catchment_gdf is not None:
    # Transform the catchment GeoDataFrame to the DEM CRS
    print(f"Original CRS: {catchment_gdf.crs}")
    catchment_gdf = catchment_gdf.to_crs(dem.crs)
    print(f"Transformed CRS: {catchment_gdf.crs}")
    
    # Extract the raster data for the catchment
    print(f"Extracting the raster data for catchment {catchment_gdf['name'].iloc[0]} ...")
    out_image, out_transform = mask(dem, [mapping(catchment_gdf.geometry.iloc[0])], crop=True)
    print(f"Raster shape: {out_image.shape}")
    print(f"Raster Nodata value: {nodata_value}")
    
    out_image = out_image.squeeze()
    print(f"Squeezed raster shape: {out_image.shape}")
    
    # Check if the raster contains nodata values
    valid_mask = out_image != nodata_value
    out_image = out_image[valid_mask]
    print(f"Valid pixel count: {out_image.size}")
    
    # Check the unique values in the catchment area
    unique_values = set(out_image)
    print(f"Unique elevation values in the catchment area: {sorted(unique_values)[:10]}")

    # Plot the elevation values in the catchment area
    plt.hist(out_image, bins=30, edgecolor='black')
    plt.title('Elevation Values in the Catchment Area')
    plt.xlabel('Elevation')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

else:
    print("No catchment GeoDataFrame provided.")


Es taucht eine größere Menge von Werten auf, die exakt null sind und das Ergebnis der mittleren Höhe beeinflussen. Da es sich offenbar um Fehler aus der Verarbeitung handelt, werden alle Werte, die exakt null sind, ausgeschlossen.

In [ ]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
import numpy as np
import matplotlib.pyplot as plt
import os

from catchments import get_catchment_gdf

Festlegen der Gebiets-ID

In [ ]:
ID = 1

catchment_gdf = get_catchment_gdf(ID)

Laden und Verarbeiten der DEM-Daten

In [ ]:
# Load the merged Copernicus DEM raster
print("Loading merged Copernicus DEM raster")
dem = rasterio.open(".../input_data/dem/dem_merged.tif")

# Setting the nodata value if not defined in the raster metadata
if dem.nodata is None:
    nodata_value = -9999
else:
    nodata_value = dem.nodata

print(f"DEM bounds: {dem.bounds}")
print(f"DEM CRS: {dem.crs}")

if catchment_gdf is not None:
    # Transform the catchment GeoDataFrame to the DEM CRS
    print(f"Original CRS: {catchment_gdf.crs}")
    catchment_gdf = catchment_gdf.to_crs(dem.crs)
    print(f"Transformed CRS: {catchment_gdf.crs}")
    
    # Extract the raster data for the catchment
    print(f"Extracting the raster data for catchment {catchment_gdf['name'].iloc[0]} ...")
    out_image, out_transform = mask(dem, [mapping(catchment_gdf.geometry.iloc[0])], crop=True)
    print(f"Raster shape: {out_image.shape}")
    print(f"Raster Nodata value: {nodata_value}")
    
    out_image = out_image.squeeze()
    print(f"Squeezed raster shape: {out_image.shape}")
    
    # Check if the raster contains nodata values
    valid_mask = (out_image != nodata_value) & (out_image != 0)
    out_image = out_image[valid_mask]
    print(f"Valid pixel count: {out_image.size}")
    
    # Check the unique values in the catchment area
    unique_values = set(out_image)
    print(f"Unique elevation values in the catchment area: {sorted(unique_values)[:10]}")

    # Plot the elevation values in the catchment area
    plt.hist(out_image, bins=30, edgecolor='black')
    plt.title('Elevation Values in the Catchment Area')
    plt.xlabel('Elevation')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

    # Calculate the mean elevation value in the catchment area
    catchment_stats = {
        'gauge_id': ID,
        'elev_mean': np.mean(out_image)
    }
    print(f"Calculated stats: {catchment_stats}")
    
    catchment_stats_df = pd.DataFrame([catchment_stats])

    #Save the extracted data to a CSV file
    print("Saving the extracted data")
    
    file_path = ".../output_data/camels_de/CAMELS_DE_topographic_attributes.csv"
    if not os.path.exists(file_path):
        catchment_stats_df.to_csv(file_path, index=False)
    else:
        catchment_stats_df.to_csv(file_path, mode='a', header=False, index=False)

else:
    print("No catchment GeoDataFrame provided.")

### Plot der Höhendaten mit einem Overlay des akuellen Einzugsgebiets

Dieser Block dient zu Veranschaulichung und Kontrolle. Da er viel Rechenzeit beansprucht, sollte er für eine schnelle Berechnung übersprungen werden, da er nicht zur Berechnung der Werte dient

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from shapely.geometry import mapping

from catchments import get_catchment_gdf

ID = 1

catchment_gdf = get_catchment_gdf(ID)

# Load the merged Copernicus DEM raster
print("Loading merged Copernicus DEM raster")
dem = rasterio.open(".../input_data/dem/dem_merged.tif")

if catchment_gdf is not None:
    #Transform the catchment GeoDataFrame to the DEM CRS
    catchment_gdf = catchment_gdf.to_crs(dem.crs)
    
    # Plot the catchment overlay on the DEM
    fig, ax = plt.subplots(figsize=(10, 10))
    show(dem, ax=ax, title="Catchment Overlay on DEM")
    catchment_gdf.plot(ax=ax, facecolor='none', edgecolor='red')
    plt.show()
else:
    print("No catchment GeoDataFrame provided.")
